In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import IPython
import requests
import dotenv

from backend.location import Location, make_grid
from backend.gmaps import get_static_map, get_distance_matrix

dotenv.load_dotenv()

In [ ]:
GMAPS_API_KEY = os.environ["GMAPS_API_KEY"]

In [ ]:
# zurich = Location(47.378, 8.540)
zurich = Location(47.378, 8.540)
# zurich = Location(45, 130)

In [ ]:
zoom = 14
grid = make_grid(zurich, zoom, size=9)

image = get_static_map(zurich, zoom, markers=grid)

IPython.display.Image(image)

In [ ]:
matrix = list(get_distance_matrix(grid, grid))

In [ ]:
len(matrix)

In [ ]:
with open("9x9matrix.json", "w") as f:
    json.dump(matrix, f)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(response.json())

In [ ]:
df["duration_sec"] = df["duration"].apply(lambda x: x if pd.isna(x) else int(x[:-1]))

In [ ]:
df

In [ ]:
df["velocity"] = df["distanceMeters"] / df["duration_sec"]

In [ ]:
df.query("~velocity.isna()").sort_values("velocity", ascending=False)